In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree

from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.Embarked = train.Embarked.fillna("NAN")
train.dropna(subset=["Embarked"], axis=0, inplace=True)
test.Fare = test.Fare.fillna(7.8)
train = train.loc[train.Fare<300, :]
titanic = pd.concat([train, test], axis=0, ignore_index=True)
titanic["Fare_Log"] = np.log1p(titanic.Fare) 

In [4]:
titanic["Cabin"] = titanic.Cabin.fillna("N")
#titanic["Cabin_Number"] = titanic.Cabin.map(lambda x: x.strip()[1:3] if len(x.split(" ")[0])==3 else x.strip()[1:4]).map(lambda x: )
titanic["IsCabin"] = titanic.Cabin.map(lambda x: 0 if x == "N" else 1)

In [5]:
titanic["Title"] = titanic.Name.map(lambda x: x.split(",")[1].split(".")[0].strip()).replace(["Capt", "Col", "Don", "Dr",
                                    "Jonkheer", "Major","Rev","Sir"], "Others").replace(["Dona", "Lady", "Mlle", "Mme",
                                                                                         "Ms", "the Countess"],"Mrs")
titanic["Surname"] = titanic.Name.map(lambda x: x.split(",")[0].strip())
titanic["Family"] = titanic.SibSp + titanic.Parch + 1
titanic["Family_Bin"] = titanic.Family.replace([2,3,4],2).replace([5,6,7,8,11], 3)
titanic["IsMarige"] = titanic.Parch.map(lambda x: 1 if x > 0 else 0)

In [6]:
titanic["Fare_Log_Bin"] = pd.cut(titanic["Fare_Log"], bins=[-0.5,1,2.4,3,4,5,7], labels=["f1","f2","f3","f4","f5","f6"])


#titanic.Age = titanic.Age.fillna(titanic.Age.median())
#titanic["Age"] = titanic.groupby(["Sex", "Pclass", "IsCabin", "Family_Bin", "Fare_Log_Bin", "Embarked", "IsMarige"])["Age"].apply(lambda x: x.fillna(x.mean()))
#titanic["Age"] = titanic.groupby(["Sex", "Pclass", "IsCabin", "Family_Bin", "Fare_Log_Bin", "Embarked"])["Age"].apply(lambda x: x.fillna(x.mean()))
#titanic["Age"] = titanic.groupby(["Sex", "Pclass", "IsCabin", "Family_Bin", "Fare_Log_Bin"])["Age"].apply(lambda x: x.fillna(x.mean()))
#titanic["Age"] = titanic.groupby(["Sex", "Pclass", "IsCabin", "Family_Bin"])["Age"].apply(lambda x: x.fillna(x.mean()))

titanic["Age_Bin"] = pd.cut(titanic.Age, bins=[-0.5,10,17,25,30,40,60,85], labels=["age1","age2","age3","age4","age5","age6","age7"])
titanic["Age_Log"] = np.log1p(titanic.Age)

In [7]:
titanic["Ticket_First"] = titanic.Ticket.map(lambda x: x[0]).replace(["4","5","6","7","8","A","L","W","9"], "4").replace("F","C")
titanic["Ticket_Len"] = titanic.Ticket.map(lambda x: len(x))

In [8]:
titanic.groupby("Ticket_First").agg({"Survived":"mean", "Pclass":"mean", "Fare":"median", "Age":"mean", "Ticket":"count"})

,Survived,Pclass,Fare,Age,Ticket
Ticket_First,,,,,
1,0.630137,1.147619,53.10000,37.833829,210
2,0.464481,2.309353,13.50000,28.140217,278
3,0.239203,2.904429,7.89580,26.082006,429
4,0.129870,2.783019,9.22500,30.583333,106
C,0.370370,2.455556,21.00000,26.095811,90
P,0.629032,1.105263,76.72920,38.463855,95
S,0.323077,2.561224,8.10415,26.514881,98


In [9]:
titanic.pivot_table(index="Ticket_First", columns="Sex", values="Survived")

Sex,female,male
Ticket_First,,
1,0.948276,0.420455
2,0.840000,0.203704
3,0.574468,0.086957
4,0.238095,0.089286
C,0.750000,0.147059
P,0.967742,0.290323
S,0.714286,0.215686


In [10]:
Id = test.PassengerId
target = train.Survived
split_index = len(train)

### MODELING

### CATBOOST

In [11]:
cat_titanic = titanic.drop(["PassengerId", "Survived", "Name", "Ticket", "Cabin", "Surname",
                             "SibSp", "Parch", "Fare", "Age_Bin", "Fare_Log_Bin", "Family", "Age_Log",
                             "Ticket_Len"], axis=1)

In [12]:
cat_titanic

,Pclass,Sex,Age,Embarked,Fare_Log,IsCabin,Title,Family_Bin,IsMarige,Ticket_First
0,3,male,22.0,S,2.110213,0,Mr,2,0,4
1,1,female,38.0,C,4.280593,1,Mrs,2,0,P
2,3,female,26.0,S,2.188856,0,Miss,1,0,S
3,1,female,35.0,S,3.990834,1,Mrs,2,0,1
4,3,male,35.0,S,2.202765,0,Mr,1,0,3
...,...,...,...,...,...,...,...,...,...,...
1301,3,male,NaN,S,2.202765,0,Mr,1,0,4
1302,1,female,39.0,C,4.699571,1,Mrs,1,0,P
1303,3,male,38.5,S,2.110213,0,Mr,1,0,S
1304,3,male,NaN,S,2.202765,0,Mr,1,0,3


In [13]:
cats = ["Pclass", "Sex", "Embarked", "IsCabin", "Title", "Family_Bin", "IsMarige", "Ticket_First"]
cat_titanic_train = cat_titanic.iloc[:split_index]
X_cat_train, X_cat_val, y_train, y_val = train_test_split(cat_titanic_train, target, test_size=0.25)
cat_titanic_test = cat_titanic.iloc[split_index:]
train_pool = Pool(X_cat_train, y_train, cat_features=cats)
val_pool = Pool(X_cat_val, y_val, cat_features=cats)
test_pool = Pool(cat_titanic_test, cat_features=cats)

In [14]:
cb = CatBoostClassifier()
p = {"iterations":[250, 550, 1000],
    "depth":[4,6,7,8],
    "l2_leaf_reg":[0, 1.5, 3.0, 5.0]}
cb.grid_search(p, Pool(cat_titanic_train, target, cat_features=cats), cv=8)

0:	learn: 0.6731648	test: 0.6729675	best: 0.6729675 (0)	total: 152ms	remaining: 37.9s
1:	learn: 0.6563493	test: 0.6556950	best: 0.6556950 (1)	total: 168ms	remaining: 20.8s
2:	learn: 0.6411646	test: 0.6380103	best: 0.6380103 (2)	total: 185ms	remaining: 15.2s
3:	learn: 0.6279345	test: 0.6222623	best: 0.6222623 (3)	total: 204ms	remaining: 12.6s
4:	learn: 0.6117256	test: 0.6091526	best: 0.6091526 (4)	total: 235ms	remaining: 11.5s
5:	learn: 0.6000338	test: 0.5969713	best: 0.5969713 (5)	total: 250ms	remaining: 10.2s
6:	learn: 0.5920625	test: 0.5882849	best: 0.5882849 (6)	total: 256ms	remaining: 8.89s
7:	learn: 0.5795953	test: 0.5757267	best: 0.5757267 (7)	total: 270ms	remaining: 8.16s
8:	learn: 0.5701580	test: 0.5636948	best: 0.5636948 (8)	total: 286ms	remaining: 7.65s
9:	learn: 0.5597476	test: 0.5533996	best: 0.5533996 (9)	total: 311ms	remaining: 7.47s
10:	learn: 0.5508429	test: 0.5441803	best: 0.5441803 (10)	total: 341ms	remaining: 7.41s
11:	learn: 0.5427550	test: 0.5347758	best: 0.5347758

98:	learn: 0.3653378	test: 0.3787739	best: 0.3777380 (90)	total: 2s	remaining: 3.06s
99:	learn: 0.3651363	test: 0.3791709	best: 0.3777380 (90)	total: 2.03s	remaining: 3.04s
100:	learn: 0.3643801	test: 0.3802446	best: 0.3777380 (90)	total: 2.06s	remaining: 3.04s
101:	learn: 0.3640984	test: 0.3801810	best: 0.3777380 (90)	total: 2.07s	remaining: 3.01s
102:	learn: 0.3639321	test: 0.3800947	best: 0.3777380 (90)	total: 2.09s	remaining: 2.98s
103:	learn: 0.3636302	test: 0.3801402	best: 0.3777380 (90)	total: 2.1s	remaining: 2.95s
104:	learn: 0.3628445	test: 0.3806958	best: 0.3777380 (90)	total: 2.12s	remaining: 2.93s
105:	learn: 0.3624599	test: 0.3807079	best: 0.3777380 (90)	total: 2.13s	remaining: 2.9s
106:	learn: 0.3615371	test: 0.3808166	best: 0.3777380 (90)	total: 2.17s	remaining: 2.9s
107:	learn: 0.3605668	test: 0.3803629	best: 0.3777380 (90)	total: 2.21s	remaining: 2.91s
108:	learn: 0.3599016	test: 0.3805600	best: 0.3777380 (90)	total: 2.24s	remaining: 2.9s
109:	learn: 0.3596052	test: 0.

194:	learn: 0.3139769	test: 0.3772000	best: 0.3760097 (173)	total: 3.9s	remaining: 1.1s
195:	learn: 0.3134565	test: 0.3771772	best: 0.3760097 (173)	total: 3.93s	remaining: 1.08s
196:	learn: 0.3133985	test: 0.3771724	best: 0.3760097 (173)	total: 3.94s	remaining: 1.06s
197:	learn: 0.3126505	test: 0.3770924	best: 0.3760097 (173)	total: 3.96s	remaining: 1.04s
198:	learn: 0.3116401	test: 0.3767931	best: 0.3760097 (173)	total: 3.99s	remaining: 1.02s
199:	learn: 0.3112948	test: 0.3768883	best: 0.3760097 (173)	total: 4s	remaining: 1s
200:	learn: 0.3108186	test: 0.3767862	best: 0.3760097 (173)	total: 4.02s	remaining: 980ms
201:	learn: 0.3093195	test: 0.3756597	best: 0.3756597 (201)	total: 4.03s	remaining: 959ms
202:	learn: 0.3091046	test: 0.3756943	best: 0.3756597 (201)	total: 4.05s	remaining: 938ms
203:	learn: 0.3080359	test: 0.3758068	best: 0.3756597 (201)	total: 4.07s	remaining: 917ms
204:	learn: 0.3073222	test: 0.3760717	best: 0.3756597 (201)	total: 4.08s	remaining: 896ms
205:	learn: 0.3058

41:	learn: 0.4315661	test: 0.4097064	best: 0.4097064 (41)	total: 1.03s	remaining: 5.11s
42:	learn: 0.4297464	test: 0.4076361	best: 0.4076361 (42)	total: 1.05s	remaining: 5.04s
43:	learn: 0.4282650	test: 0.4060148	best: 0.4060148 (43)	total: 1.08s	remaining: 5.05s
44:	learn: 0.4268643	test: 0.4043000	best: 0.4043000 (44)	total: 1.11s	remaining: 5.07s
45:	learn: 0.4248618	test: 0.4022987	best: 0.4022987 (45)	total: 1.14s	remaining: 5.06s
46:	learn: 0.4238806	test: 0.4011007	best: 0.4011007 (46)	total: 1.17s	remaining: 5.07s
47:	learn: 0.4223838	test: 0.3991268	best: 0.3991268 (47)	total: 1.2s	remaining: 5.03s
48:	learn: 0.4207284	test: 0.3974409	best: 0.3974409 (48)	total: 1.21s	remaining: 4.97s
49:	learn: 0.4191391	test: 0.3965580	best: 0.3965580 (49)	total: 1.23s	remaining: 4.93s
50:	learn: 0.4175770	test: 0.3950410	best: 0.3950410 (50)	total: 1.25s	remaining: 4.88s
51:	learn: 0.4164875	test: 0.3930840	best: 0.3930840 (51)	total: 1.27s	remaining: 4.82s
52:	learn: 0.4155930	test: 0.3923

137:	learn: 0.3728896	test: 0.3660293	best: 0.3650886 (124)	total: 3.31s	remaining: 2.69s
138:	learn: 0.3726293	test: 0.3662790	best: 0.3650886 (124)	total: 3.33s	remaining: 2.65s
139:	learn: 0.3720093	test: 0.3672049	best: 0.3650886 (124)	total: 3.35s	remaining: 2.63s
140:	learn: 0.3714660	test: 0.3669380	best: 0.3650886 (124)	total: 3.38s	remaining: 2.61s
141:	learn: 0.3711504	test: 0.3669344	best: 0.3650886 (124)	total: 3.4s	remaining: 2.59s
142:	learn: 0.3704995	test: 0.3669815	best: 0.3650886 (124)	total: 3.43s	remaining: 2.56s
143:	learn: 0.3704554	test: 0.3669765	best: 0.3650886 (124)	total: 3.44s	remaining: 2.54s
144:	learn: 0.3700943	test: 0.3669350	best: 0.3650886 (124)	total: 3.47s	remaining: 2.52s
145:	learn: 0.3700618	test: 0.3667496	best: 0.3650886 (124)	total: 3.49s	remaining: 2.49s
146:	learn: 0.3696228	test: 0.3672077	best: 0.3650886 (124)	total: 3.51s	remaining: 2.46s
147:	learn: 0.3687173	test: 0.3674811	best: 0.3650886 (124)	total: 3.55s	remaining: 2.45s
148:	learn:

...

191:	learn: 0.3503187	test: 0.4263311	best: 0.4146459 (70)	total: 4.78s	remaining: 1.44s
192:	learn: 0.3497100	test: 0.4269260	best: 0.4146459 (70)	total: 4.79s	remaining: 1.42s
193:	learn: 0.3496233	test: 0.4269284	best: 0.4146459 (70)	total: 4.83s	remaining: 1.39s
194:	learn: 0.3490480	test: 0.4273270	best: 0.4146459 (70)	total: 4.85s	remaining: 1.37s
195:	learn: 0.3487855	test: 0.4275368	best: 0.4146459 (70)	total: 4.87s	remaining: 1.34s
196:	learn: 0.3486258	test: 0.4275828	best: 0.4146459 (70)	total: 4.91s	remaining: 1.32s
197:	learn: 0.3484008	test: 0.4276134	best: 0.4146459 (70)	total: 4.92s	remaining: 1.29s
198:	learn: 0.3481852	test: 0.4275270	best: 0.4146459 (70)	total: 4.95s	remaining: 1.27s
199:	learn: 0.3481405	test: 0.4275213	best: 0.4146459 (70)	total: 4.96s	remaining: 1.24s
200:	learn: 0.3480592	test: 0.4275539	best: 0.4146459 (70)	total: 5s	remaining: 1.22s
201:	learn: 0.3479568	test: 0.4275280	best: 0.4146459 (70)	total: 5.01s	remaining: 1.19s
202:	learn: 0.3476973	te

37:	learn: 0.4400345	test: 0.4166379	best: 0.4166379 (37)	total: 952ms	remaining: 5.31s
38:	learn: 0.4379599	test: 0.4145735	best: 0.4145735 (38)	total: 979ms	remaining: 5.29s
39:	learn: 0.4366605	test: 0.4130025	best: 0.4130025 (39)	total: 997ms	remaining: 5.23s
40:	learn: 0.4353352	test: 0.4108671	best: 0.4108671 (40)	total: 1.01s	remaining: 5.17s
41:	learn: 0.4335403	test: 0.4095382	best: 0.4095382 (41)	total: 1.04s	remaining: 5.16s
42:	learn: 0.4324367	test: 0.4084483	best: 0.4084483 (42)	total: 1.05s	remaining: 5.08s
43:	learn: 0.4306437	test: 0.4067083	best: 0.4067083 (43)	total: 1.07s	remaining: 5.03s
44:	learn: 0.4288539	test: 0.4051404	best: 0.4051404 (44)	total: 1.09s	remaining: 4.96s
45:	learn: 0.4275092	test: 0.4035247	best: 0.4035247 (45)	total: 1.12s	remaining: 4.96s
46:	learn: 0.4257244	test: 0.4011051	best: 0.4011051 (46)	total: 1.15s	remaining: 4.97s
47:	learn: 0.4241383	test: 0.3988308	best: 0.3988308 (47)	total: 1.17s	remaining: 4.93s
48:	learn: 0.4233607	test: 0.398

133:	learn: 0.3794720	test: 0.3611579	best: 0.3611579 (133)	total: 3.32s	remaining: 2.87s
134:	learn: 0.3792608	test: 0.3612040	best: 0.3611579 (133)	total: 3.34s	remaining: 2.84s
135:	learn: 0.3790918	test: 0.3611963	best: 0.3611579 (133)	total: 3.36s	remaining: 2.82s
136:	learn: 0.3787083	test: 0.3607961	best: 0.3607961 (136)	total: 3.38s	remaining: 2.79s
137:	learn: 0.3784243	test: 0.3605872	best: 0.3605872 (137)	total: 3.41s	remaining: 2.77s
138:	learn: 0.3782587	test: 0.3603161	best: 0.3603161 (138)	total: 3.44s	remaining: 2.75s
139:	learn: 0.3780825	test: 0.3604039	best: 0.3603161 (138)	total: 3.46s	remaining: 2.72s
140:	learn: 0.3777665	test: 0.3605202	best: 0.3603161 (138)	total: 3.48s	remaining: 2.69s
141:	learn: 0.3776364	test: 0.3601318	best: 0.3601318 (141)	total: 3.5s	remaining: 2.66s
142:	learn: 0.3775326	test: 0.3600897	best: 0.3600897 (142)	total: 3.52s	remaining: 2.64s
143:	learn: 0.3768221	test: 0.3598104	best: 0.3598104 (143)	total: 3.54s	remaining: 2.61s
144:	learn:

231:	learn: 0.3533926	test: 0.3556115	best: 0.3556115 (231)	total: 5.7s	remaining: 442ms
232:	learn: 0.3532494	test: 0.3557777	best: 0.3556115 (231)	total: 5.73s	remaining: 418ms
233:	learn: 0.3530618	test: 0.3552066	best: 0.3552066 (233)	total: 5.75s	remaining: 393ms
234:	learn: 0.3528681	test: 0.3554435	best: 0.3552066 (233)	total: 5.78s	remaining: 369ms
235:	learn: 0.3525805	test: 0.3554158	best: 0.3552066 (233)	total: 5.8s	remaining: 344ms
236:	learn: 0.3523585	test: 0.3557056	best: 0.3552066 (233)	total: 5.82s	remaining: 319ms
237:	learn: 0.3521948	test: 0.3557740	best: 0.3552066 (233)	total: 5.83s	remaining: 294ms
238:	learn: 0.3516088	test: 0.3552278	best: 0.3552066 (233)	total: 5.87s	remaining: 270ms
239:	learn: 0.3514725	test: 0.3555501	best: 0.3552066 (233)	total: 5.89s	remaining: 246ms
240:	learn: 0.3514699	test: 0.3555501	best: 0.3552066 (233)	total: 5.91s	remaining: 221ms
241:	learn: 0.3514292	test: 0.3555501	best: 0.3552066 (233)	total: 5.93s	remaining: 196ms
242:	learn: 

{'params': {'depth': 4, 'iterations': 250, 'l2_leaf_reg': 3.0},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
      

In [21]:
CatBoostClassifier?

### SUBMIS

In [15]:
#cb.fit(Pool(cat_titanic_train, target, cat_features=cats))

In [18]:
predict = cb.predict(test_pool)

In [19]:
pd.DataFrame({"PassengerId":Id, "Survived":predict}).set_index("PassengerId").to_csv("catboost_sub.csv")